In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [7]:
drug_cost = pd.read_csv("NADAC__National_Average_Drug_Acquisition_Cost_.csv")
drug_cost = drug_cost[["NDC Description", "NADAC_Per_Unit"]]
drug_cost.columns = ["drugName", "price"]

drugcom = pd.read_csv("drugcom_train.csv")
drugcom = drugcom.iloc[:, 1:]
drugcom.columns = drugcom.iloc[0, :]
drugcom = drugcom.iloc[1:, :]

drugcom_test = pd.read_csv("drugcom_test.csv")
drugcom_test = drugcom_test.iloc[:, 1:]
drugcom_test.columns = drugcom_test.iloc[0, :]
drugcom_test = drugcom_test.iloc[1:, :]

druglib = pd.read_csv("druglib_train.csv")
druglib = druglib.iloc[:, 1:]
druglib.columns = ['drugName', 'rating', 'effectiveness', 'sideEffects', 'condition',
       'benefitsReview', 'sideEffectsReview', 'commentsReview']
druglib = druglib.iloc[1:, :]

druglib_test = pd.read_csv("druglib_test.csv")
druglib_test = druglib_test.iloc[:, 1:]
druglib_test.columns = ['drugName', 'rating', 'effectiveness', 'sideEffects', 'condition',
       'benefitsReview', 'sideEffectsReview', 'commentsReview']
druglib_test = druglib_test.iloc[1:, :]

dnames = pd.read_csv("drugNames.csv")
dnames = dnames[["DrugName", "ActiveIngredient"]]



C:\Users\Jordan Fan\Anaconda3\envs\stat159\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Processing drug names, take first word of drug name

In [3]:
#removes punctuations, numbers, hyphens, and then takes the first word of the 
def process_str(df, column):
    df2 = df.copy()
    new_column_name = "first_" + column
    new_column = df2[column].str.lower().str.replace("[^a-z\s-]", " ").str.replace("-", "").str.strip().str.split(" ").apply(lambda x: x[0])
    df2[new_column_name] = new_column
    return df2

drug_cost = process_str(drug_cost, "drugName")
dnames = process_str(dnames, "DrugName")
dnames = process_str(dnames, "ActiveIngredient")
drugcom = process_str(drugcom, "drugName")
drugcom_test = process_str(drugcom_test, "drugName")
druglib = process_str(druglib, "drugName")
druglib_test = process_str(druglib_test, "drugName")


In [4]:
druglib_full = druglib.append(druglib_test)
#If there are multiple over-the-counter names or active ingredient names associated for the drug name, then choose the most frequent one that shows up
druglib_full["active_ingredient_name"] = druglib_full.first_drugName.apply(lambda x: dnames[dnames.first_DrugName == x].first_ActiveIngredient.value_counts().index[0]
                                                           if x in dnames.first_DrugName.tolist() else x)
druglib_full["over_counter_name"] = druglib_full.first_drugName.apply(lambda x: dnames[dnames.first_ActiveIngredient == x].first_DrugName.value_counts().index[0]
                                                           if x in dnames.first_ActiveIngredient.tolist() else x)


### Unique drug names, active ingredient names, and over the counter names

In [5]:
druglib_full[druglib_full.first_drugName != druglib_full.over_counter_name]

,drugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview,first_drugName,active_ingredient_name,over_counter_name
2388,amphetamine,9,Considerably Effective,Mild Side Effects,add,Elimated restless feeling and able to sleep at...,"Taken on an empty stomach, it makes you a litt...",My doctor recommended this course of treatment...,amphetamine,amphetamine,dextroamp


Only instance in druglib where the drug name didn't match up with the over counter name was for amphetamine

In [6]:
drugName_active_ingredient = druglib_full.drugName.append(druglib_full.active_ingredient_name).unique()

### Selecting drugcom data that's in drug name or active ingredient 

In [7]:
drugcom_whole = drugcom.append(drugcom_test)
drugcom_whole = drugcom_whole[drugcom_whole.first_drugName.isin(drugName_active_ingredient)]

In [8]:
drugcom_whole["over_counter_name"] = drugcom_whole.first_drugName.apply(lambda x: dnames[dnames.first_ActiveIngredient == x].first_DrugName.value_counts().index[0]
                                                           if x in dnames.first_ActiveIngredient.tolist() else x)

In [9]:
drugcom_whole.loc[drugcom_whole.first_drugName == "amphetamine", "over_counter_name"] = "amphetamine"
drugcom_whole = drugcom_whole[drugcom_whole.over_counter_name.isin(druglib["first_drugName"])]

Since the over-the-counter name is the same for all the drug name in druglib after changing amphetamine to amphetamine, after converting the drugcom drug names to over the counter names, can just compare the over the counter names with the drugnames in druglib and only include those matches

In [10]:
drug_whole = druglib_full.append(drugcom_whole)

C:\Users\Jordan Fan\Anaconda3\envs\stat159\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [11]:
drug_whole.to_csv("drug_whole.csv")

### Drug Costs

##### Same process of filtering as drugcom 

In [8]:
drug_costs = drug_cost[drug_cost.first_drugName.isin(drugName_active_ingredient)]

AttributeError: 'DataFrame' object has no attribute 'first_drugName'

In [13]:
#grouped drugs and took minimum price 
drug_costs_min_grouped = drug_costs.groupby("first_drugName", as_index = False)["price"].min()
drug_costs_min_grouped["over_counter_name"] = drug_costs_min_grouped.first_drugName.apply(lambda x: dnames[dnames.first_ActiveIngredient == x].first_DrugName.value_counts().index[0]
                                                           if x in dnames.first_ActiveIngredient.tolist() else x)

In [14]:
drug_costs_min_grouped.loc[drug_costs_min_grouped.first_drugName == "amphetamine", "over_counter_name"] = "amphetamine"
drug_costs_min_grouped = drug_costs_min_grouped[drug_costs_min_grouped["over_counter_name"].isin(druglib["first_drugName"])]
drug_costs_min_grouped = drug_costs_min_grouped.drop("first_drugName", axis = 1)

### Merging Drug Costs with Drug Whole 

In [15]:
drug_costs_min_grouped.head()

,price,over_counter_name
0,4.73227,abilify
2,1.84436,accolate
3,2.24099,accupril
5,11.00438,aciphex
6,5.03782,actonel


In [16]:
drug_whole.head()

,active_ingredient_name,benefitsReview,commentsReview,condition,date,drugName,effectiveness,first_drugName,over_counter_name,rating,review,sideEffects,sideEffectsReview,usefulCount
1,enalapril,slowed the progression of left ventricular dys...,"monitor blood pressure , weight and asses for ...",management of congestive heart failure,NaN,enalapril,Highly Effective,enalapril,enalapril,4,NaN,Mild Side Effects,"cough, hypotension , proteinuria, impotence , ...",NaN
2,orthotricyclen,Although this type of birth control has more c...,"I Hate This Birth Control, I Would Not Suggest...",birth prevention,NaN,ortho-tri-cyclen,Highly Effective,orthotricyclen,orthotricyclen,1,NaN,Severe Side Effects,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...",NaN
3,mefenamic,I was used to having cramps so badly that they...,I took 2 pills at the onset of my menstrual cr...,menstrual cramps,NaN,ponstel,Highly Effective,ponstel,ponstel,10,NaN,No Side Effects,Heavier bleeding and clotting than normal.,NaN
4,omeprazole,The acid reflux went away for a few months aft...,I was given Prilosec prescription at a dose of...,acid reflux,NaN,prilosec,Marginally Effective,prilosec,prilosec,3,NaN,Mild Side Effects,"Constipation, dry mouth and some mild dizzines...",NaN
5,pregabalin,I think that the Lyrica was starting to help w...,See above,fibromyalgia,NaN,lyrica,Marginally Effective,lyrica,lyrica,2,NaN,Severe Side Effects,I felt extremely drugged and dopey. Could not...,NaN


In [17]:
drug_merged = drug_whole.merge(drug_costs_min_grouped, how = "left")
drug_merged.head()

,active_ingredient_name,benefitsReview,commentsReview,condition,date,drugName,effectiveness,first_drugName,over_counter_name,rating,review,sideEffects,sideEffectsReview,usefulCount,price
0,enalapril,slowed the progression of left ventricular dys...,"monitor blood pressure , weight and asses for ...",management of congestive heart failure,NaN,enalapril,Highly Effective,enalapril,enalapril,4,NaN,Mild Side Effects,"cough, hypotension , proteinuria, impotence , ...",NaN,0.10109
1,orthotricyclen,Although this type of birth control has more c...,"I Hate This Birth Control, I Would Not Suggest...",birth prevention,NaN,ortho-tri-cyclen,Highly Effective,orthotricyclen,orthotricyclen,1,NaN,Severe Side Effects,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...",NaN,NaN
2,mefenamic,I was used to having cramps so badly that they...,I took 2 pills at the onset of my menstrual cr...,menstrual cramps,NaN,ponstel,Highly Effective,ponstel,ponstel,10,NaN,No Side Effects,Heavier bleeding and clotting than normal.,NaN,NaN
3,omeprazole,The acid reflux went away for a few months aft...,I was given Prilosec prescription at a dose of...,acid reflux,NaN,prilosec,Marginally Effective,prilosec,prilosec,3,NaN,Mild Side Effects,"Constipation, dry mouth and some mild dizzines...",NaN,0.61204
4,pregabalin,I think that the Lyrica was starting to help w...,See above,fibromyalgia,NaN,lyrica,Marginally Effective,lyrica,lyrica,2,NaN,Severe Side Effects,I felt extremely drugged and dopey. Could not...,NaN,3.58681


In [21]:
drug_merged = drug_merged.drop(["first_drugName", "over_counter_name"], axis = 1)

In [1]:
drug_merged.to_csv("drugs_merged.csv")

NameError: name 'drug_merged' is not defined